In [47]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import holidays
import os
import urllib
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM
from keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score as R2_score
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
%matplotlib inline

In [2]:
# ELimina los warnings en la salida de la celda
warnings.simplefilter('ignore')

In [3]:
# Carga de datos de fichero pickle (9010, 31306) Ultimos 30 Años
data_sp = pd.read_pickle("C:/Users/Carap/OneDrive/Escritorio/Proyecto/datasets/data componentes SP500.pkl")

In [4]:
data_sp_copy = data_sp

In [5]:
# Definir Año para filtrar
start_date  =  '2014-10-08'
data_sp['date'] = data_sp_copy.index
data_sp_copy = data_sp_copy.reset_index(drop=True)

In [6]:
# Aqui se filtra
data_sp_copy['date']=pd.to_datetime(data_sp_copy['date'])
datefilter  =  (data_sp_copy['date']  >  start_date )
data_sp_copy  =  data_sp_copy.loc[datefilter] 

In [7]:
# resetear index
data_sp_copy.index = range(data_sp_copy.shape[0])

In [8]:
# guardar date en dataframe 
date = pd.DataFrame(data_sp_copy['date'])
date['date'] =  pd.to_datetime(date['date'])

In [9]:
# extraer nombres de compañias
company=[]
for i in data_sp_copy.columns:
    company.append(i[0]) 

In [10]:
company = list(dict.fromkeys(company))

In [11]:
# elimino date
removeWord = ['date']
company = [word for word in company if word not in removeWord]

In [12]:
# creo una lista con cada companias como dataframe
dataframes = []
for i in company:
    dataframes.append(data_sp_copy[i])

In [13]:
# creo la columna Date con la fecha
for i in enumerate(dataframes):
     i[1]['Date'] = date['date']

In [14]:
## Crear dataframe de tiempo con todos los dias (porque no estaban en el Accurate Quand data)
Timeframe = pd.date_range(start='10/08/2014', end='10/08/2019')
Timeframe = pd.DataFrame(Timeframe)
Timeframe.columns = ['Date']

In [15]:
# Añado los días que faltaban
for i in dataframes:
    i = pd.merge(i, Timeframe, on='Date', how='outer')
    i.sort_values(by = ['Date'], inplace = True)
    i = i.reset_index(drop=True)

In [16]:
# Crear una variable para cada dataframe
# for i, val in enumerate(dataframes): 
#     globals()["dataframe_Sep_%d"%i] = val

In [17]:
# obtengo los holidays para UnitedStates en una lista para los años 2015-16-17-18-19
years = [2014, 2015, 2016, 2017, 2018, 2019]
holidays_date = []
for i in years:
    for j in holidays.UnitedStates(years=i).items():
        holidays_date.append(str(j[0]))

In [18]:
# Se crea una columna con 1 si el dia coincide con la lista creada de dias holidays
for i in enumerate(dataframes):
    i[1].loc[i[1]['Date'].isin(holidays_date), 'Holiday'] = '1'
    i[1].loc[~i[1]['Date'].isin(holidays_date), 'Holiday'] = '0'

In [19]:
# Creo la columna Profitability ((close-open)/open*100) y Day_Week
# Si Week_day es Sabado o domingo o esa fecha es festiva ponme un 0 en el profitability porque no ha habido movimiento
for i in dataframes:
    i['Day_Week'] = i['Date'].dt.weekday_name
    i['Month'] = i['Date'].dt.month
    i['YearMonth'] = i['Date'].dt.to_period('M')
    i['Profitability'] = (i['close']-i['open'])/i['open']*100
    i.loc[(i['Day_Week'] == 'Saturday') | (i['Day_Week'] == 'Sunday') | (i['Holiday'] == 1), 'Profitability'] = 0

In [20]:
# creo la columna company con cada nombre de la compañia
j=0
for i in company:
    dataframes[j]['Company'] = i
    j += 1

In [21]:
# create list to remove some elements from the list of dataframes that do not mathc a condition
delete_dataframes = []

In [22]:
# Eliminate dataframes from list that have a number of NAs in PROFITABILITY is higher than 15%
for j,i in enumerate(dataframes):
    if i['Profitability'].isnull().mean() < 0.15:
        delete_dataframes.append(j)
dataframes = [x for j, x in enumerate(dataframes) if j in delete_dataframes]

In [23]:
# Eliminate columns that have a number of NAs higher than 25%
for i in dataframes:
    null_columns=list(i.columns[i.isnull().any()])

    for j in null_columns:
        if i[j].isnull().mean() < 0.25:
            pass
        else: 
            i = i.drop(columns=[j])

In [24]:
# Create a list of false predictors
false_predictors = ['open', 'high', 'low', 'close', 'bid', 'ask']

In [25]:
# Eliminate the columns that have the false predictors
j = 0
for i in dataframes[j].columns:
    for x in false_predictors:
        if i==x:
            dataframes[j] = dataframes[j].drop(columns=[i])
j += 1

# TODO BIEEEEEEN

# Probar KMeans

In [136]:
test = dataframes.copy()

In [109]:
all_dates = test[0]['Date'].values

In [88]:
len(all_company)

642

In [160]:
df_Profitability = pd.DataFrame(data= {'Date':all_dates})

In [161]:
df_Volumen = pd.DataFrame(data= {'Date':all_dates})

In [162]:
all_profitability = []
all_volumen = []
all_company = []
for i in test:
    all_profitability.append(i['Profitability'])
    all_volumen.append(i['volume'])
    all_company.append(i['Company'])

In [163]:
j = 0
for i in all_profitability:
    df_Profitability[all_company[j][0]+'_Profitability'] = i
    j += 1

In [164]:
corr_matrix_Profitability = df_Profitability.corr()

In [165]:
j = 0
for i in all_volumen:
    df_Volumen[all_company[j][0]+'_Volumen'] = i
    j += 1

In [166]:
corr_matrix_Volumen = df_Volumen.corr()

In [169]:
corr_matrix_Volumen.shape

(642, 642)

In [170]:
corr_matrix_Profitability.shape

(642, 642)

In [172]:
corr_matrix_Profitability

,A UN Equity_Profitability,AA UN Equity_Profitability,AAL UW Equity_Profitability,AAP UN Equity_Profitability,AAPL UQ Equity_Profitability,AAPL UW Equity_Profitability,ABBV UN Equity_Profitability,ABC UN Equity_Profitability,ABMD UW Equity_Profitability,ABT UN Equity_Profitability,...,XLNX UW Equity_Profitability,XOM UN Equity_Profitability,XRAY UW Equity_Profitability,XRX UN Equity_Profitability,XYL UN Equity_Profitability,YUM UN Equity_Profitability,ZBH UN Equity_Profitability,ZION UQ Equity_Profitability,ZION UW Equity_Profitability,ZTS UN Equity_Profitability
A UN Equity_Profitability,1.000000,0.340370,0.351605,0.215527,0.456718,0.456754,0.405378,0.305005,0.359000,0.565315,...,0.446854,0.367614,0.375366,0.351297,0.470630,0.393016,0.482097,0.400161,0.400324,0.461060
AA UN Equity_Profitability,0.340370,1.000000,0.270937,0.145521,0.340239,0.339965,0.182478,0.132676,0.149293,0.244495,...,0.292864,0.331362,0.206031,0.285582,0.382380,0.203182,0.218072,0.351074,0.350833,0.166687
AAL UW Equity_Profitability,0.351605,0.270937,1.000000,0.211117,0.309218,0.309097,0.207578,0.271847,0.186656,0.311886,...,0.331628,0.194349,0.248072,0.289560,0.334540,0.251280,0.256680,0.320333,0.320470,0.250859
AAP UN Equity_Profitability,0.215527,0.145521,0.211117,1.000000,0.241919,0.242257,0.215689,0.215553,0.174407,0.252047,...,0.181266,0.207241,0.171125,0.221545,0.244197,0.255816,0.210507,0.250884,0.251284,0.205300
AAPL UQ Equity_Profitability,0.456718,0.340239,0.309218,0.241919,1.000000,0.999948,0.287155,0.246205,0.313340,0.430679,...,0.446084,0.323669,0.284935,0.322972,0.358499,0.335721,0.333243,0.328941,0.328781,0.360031
AAPL UW Equity_Profitability,0.456754,0.339965,0.309097,0.242257,0.999948,1.000000,0.287220,0.246072,0.313360,0.430694,...,0.445817,0.322879,0.285012,0.322610,0.358518,0.335833,0.333242,0.328134,0.328089,0.360098
ABBV UN Equity_Profitability,0.405378,0.182478,0.207578,0.215689,0.287155,0.287220,1.000000,0.343289,0.304204,0.452105,...,0.252635,0.263342,0.290532,0.215354,0.292000,0.267152,0.414792,0.238891,0.238980,0.434584
ABC UN Equity_Profitability,0.305005,0.132676,0.271847,0.215553,0.246205,0.246072,0.343289,1.000000,0.234735,0.349228,...,0.208619,0.200328,0.294930,0.257086,0.213064,0.152936,0.328139,0.192495,0.192337,0.318419
ABMD UW Equity_Profitability,0.359000,0.149293,0.186656,0.174407,0.313340,0.313360,0.304204,0.234735,1.000000,0.407657,...,0.295398,0.157936,0.248349,0.183997,0.250906,0.273884,0.345671,0.199364,0.199391,0.318296
ABT UN Equity_Profitability,0.565315,0.244495,0.311886,0.252047,0.430679,0.430694,0.452105,0.349228,0.407657,1.000000,...,0.365667,0.332795,0.406327,0.299090,0.404571,0.383580,0.558413,0.333041,0.333071,0.476911


In [182]:
 df_correlations = pd.merge(corr_matrix_Volumen, corr_matrix_Profitability, right_index=True)

MergeError: Must pass left_on or left_index=True

In [180]:
df_correlations.shape

(0, 1284)

In [181]:
df_correlations.head()

,A UN Equity_Volumen,AA UN Equity_Volumen,AAL UW Equity_Volumen,AAP UN Equity_Volumen,AAPL UQ Equity_Volumen,AAPL UW Equity_Volumen,ABBV UN Equity_Volumen,ABC UN Equity_Volumen,ABMD UW Equity_Volumen,ABT UN Equity_Volumen,...,XLNX UW Equity_Profitability,XOM UN Equity_Profitability,XRAY UW Equity_Profitability,XRX UN Equity_Profitability,XYL UN Equity_Profitability,YUM UN Equity_Profitability,ZBH UN Equity_Profitability,ZION UQ Equity_Profitability,ZION UW Equity_Profitability,ZTS UN Equity_Profitability
